## EPA Product System Hackathon
OK, it's May 15- we have until May 30 to get something sweee-e-et finished.  What's it going to be?

Here's the plan:
 * make a forest of fragments from the spreadsheet product models
 * generate list of interior nodes and cutoffs (via automatic traversal)
 * manually process "laundry list" terminations
   - get USEEIO working
   - use USLCI
   - use Ecoinvent
   - use ELCD
 * automatically terminate cutoff fragments to various processes, by scenario
 
First task is to read the spreadsheets, and build the fragments

## First Task: validate spreadsheets
The only potentially invalid information in the spreadsheets is the parent-child relationship between parts and assemblies; MSrocka announced that the initial commit failed this validation. So here is a check.

In [1]:
import os
import xlrd

In [2]:
fg_dir = '/data/GitHub/2019/LCAproductsystemassembly_1/data-foreground/'
bg_dir = '/data/GitHub/2019/LCAproductsystemassembly_resources/data-background/'


class DuplicateAssembly(Exception):
    pass


In [3]:
from antelope_catalog.providers.xl_dict  import XlDict

In [4]:
def validate_sheet(sheet):
    """
    checks whether the sheet includes references to unrecognized assemblies; returns the names of the 
    unrecognized assemblies
    """
    xld = XlDict(sheet)
    seen = set()
    dups = set()
    for i, row in xld.iterrows():
        parent = row['Next Assembly']
        part = row['Part Number']
        if parent is not None:
            if parent not in seen:
                yield parent
            if parent in dups:
                raise DuplicateAssembly('%s (row %d)' % (parent, i))
                
        if part in seen:
            dups.add(part)
            print('!Duplicate part: %s (row %d)' % (part, i))
            
        else:
            seen.add(part)
            

In [5]:
def validate_workbook(book):
    all_m = set()
    for n in book.sheet_names():
        s = book.sheet_by_name(n)
        m = set(k for k in validate_sheet(s))
        print('%s: %s' % (n, m))
        all_m |= m
    return all_m
    

In [6]:
def validate_folder(folder):
    asm = set()
    mis = set()
    xlss = [k for k in os.listdir(folder)]
    for xls in xlss:
        try:
            xl = xlrd.open_workbook(os.path.join(folder, xls))
        except xlrd.XLRDError:
            continue
        print('Workbook: %s' % xls)
        print('Assemblies: %s' % xl.sheet_names())
        asm |= set(xl.sheet_names())
        
        m = validate_workbook(xl)
        print('Missing: %s\n' % m)
        mis |= m
    print('Cross-References: %s' % (asm & mis))
    print('Globally Missing: %s' % (mis - asm))


In [7]:
# updated-- we see the _c file has no validation errors
validate_folder(fg_dir)

Workbook: NLG F18EFG ABL.xlsx
Assemblies: ['74A430500-1017', '74A430700-1009', '74A430800-1021']
!Duplicate part: NAS516-1A (row 46)
!Duplicate part: NAS516-1A (row 54)
!Duplicate part: M83461/1-010 (row 67)
!Duplicate part: AS5986-04 (row 69)
!Duplicate part: NAS516-1A (row 70)
!Duplicate part: MS20427M2C5 (row 134)
!Duplicate part: M23053/6-105-0 (row 173)
!Duplicate part: M23053/6-105-0 (row 176)
!Duplicate part: NAS516-1A (row 248)
!Duplicate part: MS24665-153 (row 251)
!Duplicate part: MS24665-302 (row 252)
!Duplicate part: NAS516-1A (row 268)
!Duplicate part: 74A450914-2003 (row 281)
!Duplicate part: 74A450982-2003 (row 282)
!Duplicate part: 74A430912-2001 (row 294)
!Duplicate part: 74A430979-2003 (row 309)
!Duplicate part: 74A430979-2003 (row 312)
!Duplicate part: 74A430954-2001 (row 319)
!Duplicate part: M83461/1-010 (row 329)
!Duplicate part: AS5986-04 (row 333)
!Duplicate part: MS24665-153 (row 356)
!Duplicate part: MS24665-302 (row 357)
!Duplicate part: ST3M404C4 (row 362)
!

## OK, on to the fragment building
The Foreground interface allows us to create fragments, and the Qdb allows us to use canonical quantities by name.

We observe that the spreadsheets include numerous duplicate parts, but no duplicate assemblies.  This means we don't need to worry about creating re-usable sub-fragments or checking them for consistency.  

In [8]:
from antelope_catalog import LcCatalog

In [9]:
PROJECT_DIR = '/data/GitHub/2019/EPA_PSM_antelope/'
CAT = os.path.join(PROJECT_DIR, 'catalog')

In [10]:
cat = LcCatalog(CAT)

Loading JSON data from /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json:
local.qdb: /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
6 new flow entities added (6 total)
25 new quantity entities added (25 total)


In [11]:
cat.show_interfaces()

local.qdb [basic, index, quantity]


In [12]:
# cat.get_resource('local.data.GitHub.2019.EPA_PSM_antelope.catalog.psm-hackathon').remove_archive()
fg = cat.create_foreground('psm-hackathon')

local.data.GitHub.2019.EPA_PSM_antelope.catalog.psm-hackathon: /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
Loading /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
0 new fragment entities added (0 total)


In [13]:
xls = xlrd.open_workbook(os.path.join(fg_dir, 'MLG F18EFG ABL_c.xlsx'))
xl = xls.sheet_by_name(xls.sheet_names()[0])
# xld = XlDict(xl)

In [14]:
'''
Define fragment:
 o- a "configurable exchange" 
 o- core elements are a flow and a direction
 o- optional elements: 
    - parent node (default None => reference flow)
    - termination node (default None => cutoff flow)
    - exchange value (ratio between parent node activity level and terminal flow magnitude; default 1.0)
 o- configurable elements: 
    - termination node
    - exchange value 

What's happening below is simple and general:
 - each row designates a unique fragment and represents a (non-unique) flow
 - determine and retrieve flow based on part number
   = if flow does not exist, create
 - determine and retrieve parent based on next assembly
   = if parent does not exist or is None, fragment becomes reference fragment and gets a name
   = if name is already taken, that tells us there's duplication happening

Foreground interface needs to provide:
 - way to create flows and fragments
 - way to retrieve flows and fragments by reference
 - generate existing fragments with flow (opt. direction)

What about duplication?  Here are some thoughts:
 - multiple fragments that generate the same flow are fine and encouraged- that is how fragments become useful
 - "named" fragments should be synonymous with (or maybe a subset of?) "reference fragments" i.e. fragments with 
   no parent node
   * dev question: is it operable to 'name' fragments on creation, e.g. bypassing foreground.name_fragment()?
     (should [RFC 2119 "SHALL"] work; needs unit test)
 - In the present case, duplication of fragment subtrees is not a technical problem if it happens within a fragment
   subtree, but it does indicate a design problem and should probably be detected.  It seems like an error if 
   two different subfragments produce the same assembly- if the two fragments are different, the assemblies should be
   different, and if they are the same- they should be represented by the same reference fragment. 
   
   If a new record (a) already has a matching fragment and (b) that fragment has child fragments, then it 
   is a duplicate assembly. Two different approaches to that:
    = assume the existing fragment is correct and use it, [somehow] ignoring child records
    = check to see whether the existing fragment is the same as the duplicative records and raise an exception if 
      it isn't
   
   The current spreadsheet seems to not have duplication of subassemblies, but we will plan to catch them and 
   raise an exception.
 
'''

def _row_to_flow(foreground, row):
    """
    This function is specific to the EPA foregrounds and should be the final disposition of the row dict. 
    Part Number is popped to retrieve flow.  Part Name is popped to set flow name.
    """
    if 'Next Assembly' in row:
        raise AttributeError('"Next Assembly" should be popped')
    flow_ref = row.pop('Part Number')
    if flow_ref is None:
        raise ValueError('Part number cannot be none (row %d)' % i)
    flow_name = row.pop('Part Name')
    if flow_name is None:
        flow_name = '%s ASSEMBLY' % flow_ref
    flow = foreground[flow_ref]
    if flow is None:
        flow = foreground.new_flow(flow_name, 'Number of items', external_ref=flow_ref, **row)
        if foreground[flow_ref] is not flow:
            raise KeyError('Flow ref %s failed to properly register' % flow_ref)
    return flow


def _new_reference_fragment(foreground, flow, **kwargs):
    # create a new parent fragment
    parent_ref = '%s ASSEMBLY' % flow.external_ref
    if foreground[parent_ref] is not None:
        _ac = 0
        while True:
            parent_ref = '%s ASSEMBLY alt %d' % (flow.external_ref, _ac)
            if foreground[parent_ref] is not None:
                _ac += 1
                continue
            break
    frag = foreground.new_fragment(flow, 'Output', Name=parent_ref, **kwargs)
    foreground.name_fragment(frag, parent_ref)
    return frag


def _process_row(foreground, i_row, source):
    i, row = i_row
    parent_num = row.pop('Next Assembly')
    try:
        amount = float(row.pop('QNA'))
    except (TypeError, ValueError):
        amount = 1.0
    flow = _row_to_flow(foreground, row)
    # print('%s: %s' % (source, flow))
        
    exis = [k for k in foreground.fragments_with_flow(flow) if len([c for c in k.child_flows]) > 0]
    if len(exis) > 0:
        raise DuplicateAssembly('(row %d) flow: %s' % (i, flow.link))  # handle this if it comes up
        
    if parent_num is None:
        return _new_reference_fragment(foreground, flow, value=amount, Source=source)

    else:
        try:
            parent = next(k for k in foreground.fragments_with_flow(parent_num))
        except StopIteration:
            print('Bad parent reference: %s; creating new reference fragment' % parent_num)
            return _new_reference_fragment(foreground, flow, value=amount, Source=source)
                    
    frag = foreground.new_fragment(flow, 'Input', parent=parent, value=amount, Source=source)
    return


def fragment_from_xl_sheet(foreground, sheet):
    """
    This function makes the following assumptions:
     - every record corresponds to a distinct fragment
     - every distinct 'Part Number' corresponds to a distinct flow
     - every part-number flow is measured in 'Number of items' and every exchange value is 1.0 item
     - every assembly is unique and none are re-used
    The 'Part Number' is the external reference used to retrieve the flow (which could be a part or an assembly). 
    The phrase '<Part Number> ASSEMBLY' is the external reference of the named fragment that produces the flow.
    returns: a top-level fragment.  Every sub-assembly is named after its part number; leaf fragments are unnamed 
    and may be duplicated.
    """
    refs = []
    xl = XlDict(sheet)
    for row in xl.iterrows():
        source = '%s|%d' % (sheet.name, row[0]+1)
        ref = _process_row(foreground, row, source)
        if ref is not None:
            refs.append(ref)

    return refs
            
            

In [15]:
frags = fragment_from_xl_sheet(fg, xl)

In [16]:
frags[0].show_tree()

   -<--O   da8c7 [       1 Item(s)] 74A400500-1017 ASSEMBLY
    [   1 unit] 74A400500-1017 ASSEMBLY
       | -<----: 02da7 (       1 Item(s)) NUT,PLAIN,HEXAGON
       | -<----: 1d13e (       1 Item(s)) BOLT
       | -<----: 2fe84 (       1 Item(s)) NUT
       | -<----: 34ee1 (       1 Item(s)) BOLT-RETAINER
       | -<----: 3ebda (       2 Item(s)) WASHER, FLAT
       | -<----: 3f544 (       2 Item(s)) NUT, PLAIN
       | -<----: 3f8c7 (       1 Item(s)) COTTER PIN
       | -<----: 64161 (       1 Item(s)) BOLT, SHOULDER
       | -<----: 6cccb (       1 Item(s)) NUT
       | -<----: 6f434 (       1 Item(s)) BOLT, SHOULDER
       | -<----: 7401e (       1 Item(s)) PIN, COTTER
       | -<----: 827fc (       1 Item(s)) BOLT, SHOULDER
       | -<----: 88a8b (       1 Item(s)) SPACER, SLEEVE
       | -<----: 89cf4 (       1 Item(s)) PLATE
       | -<----: 90d83 (       2 Item(s)) WASHER, FLAT
       | -<----: 93fad (       1 Item(s)) STUD
       | -<----: 99d82 (       1 Item(s)) WASHER, FL

The fragment is designed to function as a self-aggregating unit process.  The 'inventory' command will report its aggregated inputs and outputs for a unit flow of the reference fragment.

In [18]:
([str(k) for k in frags[0].inventory()])

[' Input:     [10 [Item(s)]]     FITTING, LUB. []',
 ' Input:     [7 [Item(s)]]     INSERT []',
 ' Input:     [4 [Item(s)]]     BUSHING []',
 ' Input:     [4 [Item(s)]]     WASHER, FLAT []',
 ' Input:     [4 [Item(s)]]     LOCKWIRE []',
 ' Input:     [4 [Item(s)]]     BUSHING []',
 ' Input:     [4 [Item(s)]]     SCREW, SHOULDER []',
 ' Input:     [4 [Item(s)]]     COTTER PIN []',
 ' Input:     [4 [Item(s)]]     BUSHING []',
 ' Input:     [3 [Item(s)]]     NUT []',
 ' Input:     [3 [Item(s)]]     PIN, COTTER []',
 ' Input:     [3 [Item(s)]]     WASHER, KEY, LWR []',
 ' Input:     [3 [Item(s)]]     VALVE, AIR, HIGH P []',
 ' Input:     [3 [Item(s)]]     WASHER []',
 ' Input:     [3 [Item(s)]]     NUT []',
 ' Input:     [2 [Item(s)]]     BOLT []',
 ' Input:     [2 [Item(s)]]     LOCKWIRE []',
 ' Input:     [2 [Item(s)]]     LOCKING DEVICE []',
 ' Input:     [2 [Item(s)]]     BUSHING, BELLCRANK []',
 ' Input:     [2 [Item(s)]]     BUSHING,PIVOT,PTFE []',
 ' Input:     [2 [Item(s)]]     WAS

In [19]:
f1 = fragment_from_xl_sheet(fg, xls.sheet_by_name(xls.sheet_names()[1]))
[k for k in f1.inventory()]

DuplicateAssembly: (row 12) flow: local.data.GitHub.2019.EPA_PSM_antelope.catalog.psm-hackathon/74A400561-1005

### Duplicate Assemblies
OK, so now we see an interesting situation: the same subassembly, in this case `74A400561-1005`, is used in two different top-level assemblies.  We created the `DuplicateAssembly` exception to catch this, and now it has.

Just to be sure: how does `msrocka`'s code deal with this? ans: it doesn't; the `74A00561-1005` assembly in his toy repo has all its exchanges duplicated. (presumably he has corrected this later)

How do _we_ want to deal with it? Well, here are two ways:
 1. do nothing; ignore the exception. This will lead to the assembly being replicated as a subfragment tree every time it appears, but will still be correct.
 2. Fork out the assembly into a new reference fragment and reuse it. Doing this faces the possibility that the two subassembly instances are not identical.  
 
To deal with this second problem robustly, we want to screen the incoming "second" instance and verify that it is indeed identical to the "first".  Two ways occur to me to do that: (a) assume it's correct and check for discrepancies, line-by-line; (b) build two separate fragments and then check later to see if they are identical.

This is, unfortunately, a graph isomorphism problem.  It is surely something I would want the interface to take care of, though.

My approach to this will be to:
 - only look for duplicates within the sheet
 - build fragments including duplicated subassemblies
 - after the fact, hunt for fragments that produce the same flow
 - (*shudder*) determine if they are isomorphic
 - if so, branch them into separate reference fragments and reuse them

I'll do this in the class and not in a notebook.